In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import scipy.io
from sklearn.preprocessing import OneHotEncoder

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = scipy.io.loadmat("data_temp/S1-ADL1", mdict={'filled_features':'features', 'labels':'labels'})

In [3]:
features = x['filled_features']
print("Features: ", features.shape, type(features))
print(features)

Features:  (45810, 113) <class 'numpy.ndarray'>
[[ -43.  971. -339. ...   29.  -24.  165.]
 [ -33.  957. -347. ...    5.  -33.  165.]
 [ -35.  966. -363. ...   32.   47.  165.]
 ...
 [-313.  905.  656. ...   15.  -44.  -76.]
 [-232.  959.  695. ...   19.  -10.  -76.]
 [  -9.  950.  791. ...    2.   70.  -76.]]


In [5]:
labels = x['labels'][:,0]
print("Labels: ", labels.shape, type(labels), "\n", labels)
onehot_encoder = OneHotEncoder(sparse=False)
labels = onehot_encoder.fit_transform(x['labels'][:,0].reshape(-1, 1))
#labels = tf.one_hot(x['labels'][:,0])
print("\nOne hot encoded labels: ", labels.shape, type(labels), "\n", labels)

Labels:  (45810,) <class 'numpy.ndarray'> 
 [1 1 1 ... 2 2 2]

One hot encoded labels:  (45810, 5) <class 'numpy.ndarray'> 
 [[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


## Dataset definition ##
Here we take time windows of 50 samples, with a sliding step (stride) of 25 samples, and each sample is stored as an element of the Dataset.

In [6]:
window_size = 50
stride = window_size / 2
num_features = features.shape[1]
samples, classes = labels.shape
print("Number of samples: ", samples, "\nNumber of classes: ", classes)

Number of samples:  45810 
Number of classes:  5


In [8]:
windows = int(samples // stride) - 1
print("Number of windows: ", windows)

Number of windows:  1831


Dataset should have size 1831 x 50 x 113

In [9]:
cube = np.zeros([windows, window_size, num_features])
cube.shape

(1831, 50, 113)

In [10]:
lab_cum = np.zeros([windows,classes])
lab_cum.shape

(1831, 5)

In [11]:
for w in range(windows):
    index = int(w * stride)
#    print(w, index)
    cube[w,:,:] = features[index:index+window_size, :]
    l = labels[index:index+window_size,:] # shape 50 x 5
    lab_cum[w,:] = np.sum(l, axis=0) / window_size
#    print(np.sum(l, axis=0), lab_cum[w])

In [12]:
cube_t = tf.convert_to_tensor(cube)
print('cube_t: ', type(cube_t), cube_t.shape)
cube_d = tf.data.Dataset.from_tensor_slices(cube_t)
print('cube_d: ', type(cube_d))

cube_t:  <class 'tensorflow.python.framework.ops.Tensor'> (1831, 50, 113)
cube_d:  <class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


In [14]:
print('cumulative labels: ', lab_cum.shape, type(lab_cum), "\n", lab_cum)

cumulative labels:  (1831, 5) <class 'numpy.ndarray'> 
 [[0.   1.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.  ]
 ...
 [1.   0.   0.   0.   0.  ]
 [1.   0.   0.   0.   0.  ]
 [0.52 0.   0.48 0.   0.  ]]


'lab_cum' represents the percentage of samples belonging to each class. To perform classification in a more classical sense, we set to 1 the activity with the highest value and to zero the others.

In [16]:
tf.one_hot(np.argmax(lab_cum), classes)

<tf.Tensor 'one_hot_1:0' shape=(1831,) dtype=float32>

# CLASSIFICATION #

In [12]:
# parameters
batch_size = 6

In [13]:
# placeholders
X = tf.placeholder(tf.float32, shape=[None, window_size, num_features], name='input')
y = tf.placeholder(tf.float32, shape=[None, 1], name='label')

In [32]:
# layers
# (batch, 50, 113) --> (batch, 50, 113)
conv_1 = tf.layers.conv1d(inputs=X,
                          filters=113,
                          kernel_size=5,
                          strides=1,
                          padding='same',
                          activation=tf.nn.relu,
                          data_format='channels_last')


logits = tf.layers.dense(inputs=conv_1, units=classes)

In [33]:
# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=lab_cum, logits=logits))

In [34]:
# optimizer
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

ValueError: Dimension size must be evenly divisible by 50 but is 1 for 'gradients_4/softmax_cross_entropy_with_logits_3/Reshape_2_grad/Reshape' (op: 'Reshape') with input shapes: [?,50], [1] and with input tensors computed as partial shapes: input[1] = [1].

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((features,labels))
print(type(dataset))

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


In [16]:
# session

init = tf.global_variables_initializer()
epochs = 100

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(epochs):
        
        iterator = dataset.make_one_shot_iterator()
        batch_x, batch_y = iterator.get_next()
        
        sess.run(train, feed_dict={X:'batch_x', y:'batch_y'})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))

            acc = tf.reduce_mean(tf.cast(matches, tf.float32))

            print(sess.run(acc,feed_dict={X:batch_x, y:batch_y}))
            print('\n')

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.For reference, the tensor object was Tensor("IteratorGetNext:0", shape=(113,), dtype=float64) which was passed to the feed with key Tensor("input:0", shape=(?, 50, 113), dtype=float32).